In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
coding_size=30

In [ ]:
generator=keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[coding_size]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(28*28, activation="sigmoid"),
    keras.layers.Reshape([28,28])])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
discriminator=keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid")])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
gan=keras.models.Sequential([generator, discriminator])

In [ ]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable=False

In [ ]:
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [ ]:
batch_size=32
dataset=tf.data.Dataset.from_tensor_slices(x_train).shuffle(1000)
dataset=dataset.batch(batch_size, drop_remainder=True)

In [ ]:
def train_gan(gan, dataset, batch_size, coding_size, n_epochs=5):
    generator, discriminator=gan.layers
    for epoch in range(n_epochs):
        for X_batch in dataset:
          X_batch = tf.cast(X_batch, tf.float32)
          noise=tf.random.normal(shape=[batch_size, coding_size])
          generated_images=generator(noise)
          generated_images = tf.cast(generated_images, tf.float32)
          X_fake_and_real=tf.concat([generated_images, X_batch], axis=0)
          y1=tf.constant([[0.]]*batch_size+[[1.]]*batch_size)
          discriminator.trainable=True
          discriminator.train_on_batch(X_fake_and_real, y1)
            # training generator
          noise=tf.random.normal(shape=[batch_size, coding_size])
          y2=tf.constant([[1.]]*batch_size)
          discriminator.trainable=False
          gan.train_on_batch(noise, y2)

In [ ]:
train_gan(gan, dataset, batch_size, coding_size, n_epochs=5)